In [22]:
%load_ext autoreload
%autoreload 2

In [23]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [33]:
from factue.methods.llm_calls import load_template_list, load_template_parts
for x in load_template_list(job='persuasion', step="explain", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Guilt_by_Association"
"Doubt"
"Appeal_to_Hypocrisy"
"Questioning_the_Reputation"
"Flag_Waving"
"Appeal_to_Authority"
"Appeal_to_Popularity"
"Appeal_to_Values"
"Appeal_to_Fear-Prejudice"
"Straw_Man"
"Red_Herring"
"Whataboutism"
"Appeal_to_Pity"
"Causal_Oversimplification"
"False_Dilemma-No_Choice"
"Consequential_Oversimplification"
"False_Equivalence"
"Slogans"
"Conversation_Killer"
"Appeal_to_Time"
"Loaded_Language"
"Obfuscation-Vagueness-Confusion"
"Exaggeration-Minimisation"
"Repetition"


(ChatPromptValue(messages=[SystemMessage(content='You are a classifier for persuasion technique called Name Calling - Labeling.\nInput: a chunk of text\nTask: Detect if a specific technique called Name Calling - Labeling was explicitly used in the input.\nOutput: JSON file with 2 fields:\n- description - describes if and how Name Calling - Labeling technique was used.\n- verdict - output value \'true\' if you have high confidence that Name Calling - Labeling technique was used in the input text and \'false\' otherwise.\n\nFollow the definition of the Name Calling - Labeling technique:\na form of argument in which loaded labels are directed at an individual or a group, typically in an insulting or demeaning way. Labelling an object as either something the target audience fears, hates, or on the contrary finds desirable or loves. This technique calls for a qualitative judgement that disregards facts and focuses solely on the essence of the subject being characterized. This technique is i

In [37]:
import pandas as pd
from pathlib import Path
import os
import json 


root = Path("data/llm_output/persuasion/explain")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]


df['expected_output'] = df.apply(lambda row: json.dumps({
    'description': row['explanation'][-1],
    'verdict': bool(row['gold'])
}), axis=1)
df

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_properties,is_valid,explanation,think_content,updated_definition,source_file,error,status,split,expected_output
0,pl_eu_12_06_2024_n01.txt,0,135,PL,Wicemarszałek Włodzimierz Czarzasty:\nBardzo p...,False,[],train,[],[],...,[None],[True],[The input text does not contain any elements ...,[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1,pl_defence_22_05_2024_n01.txt,3608,3857,PL,Pan wspominał o ABCS – Integrated Battle Comma...,False,[],train,[],[],...,[None],[True],[The input text does not contain any elements ...,[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
2,pl_current_affairs_09_01_2025_n03.txt,1808,1984,PL,(Poseł Anna Gembicka: Decyzja została zmienion...,True,[Repetition],train,[],[],...,[None],[True],[The input text does not contain any instances...,[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
3,pl_abortion_11_04_2024_n08.txt,2952,3136,PL,"Czy czas najwyższy, by w końcu dopuścić\ndekry...",True,"[False_Equivalence, Loaded_Language, Whatabout...",train,[],[],...,[None],[True],[The input text does not employ the Guilt by A...,[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
4,pl_eu_12_06_2024_n01.txt,1277,2304,PL,"Prezentowany na posiedzeniu projekt ustawy, op...",False,[],train,[],[],...,[None],[True],[The input text discusses a proposed law aimed...,[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text discusses a pr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,pl_eu_12_06_2024_n01.txt,137,202,PL,Poseł Sprawozdawca\nDanuta Jazłowiecka:\nDzięk...,False,[],train,[],[],...,[None],[True],[The input text does not contain any loaded la...,[None],[],data/llm_output/persuasion/explain/GPT_4O_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
538,pl_current_affairs_09_01_2025_n01.txt,6563,7980,PL,Świadczenia z programu „Aktywny rodzic” ZUS\nw...,False,[],train,[],[],...,[None],[True],[The input text does not contain any loaded la...,[None],[],data/llm_output/persuasion/explain/GPT_4O_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
539,pl_defence_22_05_2024_n01.txt,2925,3108,PL,(Poseł Barbara Bartuś: Skandal.)\nSzczególnie ...,True,"[Consequential_Oversimplification, Loaded_Lang...",train,[],[],...,[None],[True],[The input text does not contain any loaded la...,[None],[Name Calling - Labeling is a form of argument...,data/llm_output/persuasion/explain/GPT_4O_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
540,pl_current_affairs_09_01_2025_n02.txt,6592,7615,PL,Sekretarz Stanu\nw Ministerstwie Klimatu i Śro...,True,"[Conversation_Killer, Repetition]",train,[],[],...,[None],[True],[The input text does not contain any instances...,[None],[],data/llm_output/persuasion/explain/GPT_4O_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."


In [61]:
SYSTEM_PART_NAME = "system"
USER_PART_NAME = "user"

def build_messages(text,expected_output,job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"):
    prompt_layout, prompt_content = load_template_parts(
        job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"
    )

    json_line = {
            "messages": [
                {"role": "system", "content": prompt_layout[SYSTEM_PART_NAME].format(technique_name=prompt_content['technique_name'],
                                                                                     technique_desc=prompt_content['technique_desc'],
                                                                                     input_example = prompt_content['input_example'],
                                                                                     output_example = prompt_content['output_example'])},
                {"role": "user", "content": prompt_layout[USER_PART_NAME].format(text = text)},
                {"role": "assistant", "content": expected_output}
            ]
        }
    return json_line
    # return(json.dumps(json_line, ensure_ascii=False) + "\n")


In [56]:
prompt_layout, prompt_content = load_template_parts(
        job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"
    )

In [57]:
prompt_layout['user'].format(text = 'hhh')

'hhh'

In [64]:
# Write DataFrame to JSONL
with open("data/finetuning/explain2detect_pers_v1.jsonl", "w", encoding="utf-8") as file:
    for _, row in df.iterrows():
        json_line = build_messages(text = row['text'],expected_output = row['expected_output'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")